In [1]:
import wave_networks.test as test

test.hello_world()

Hello World


In [2]:
import dotenv
import wandb
import os

dotenv.load_dotenv(os.path.join('/homes/55/bwilop/wsg/private/', 'vscode-ssh.env'))
api_key = os.getenv('WANDB_API_KEY')
wandb.login(key=api_key)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: benwilop (benwilop-rwth-aachen-university). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /homes/55/bwilop/.netrc


True

In [3]:
# https://soundata.readthedocs.io/en/latest/source/tutorial.html
import soundata
urbansound8k = soundata.initialize('urbansound8k', data_home='/homes/55/bwilop/gdl/ablation_study_wavelet_networks/data')
# urbansound8k.download()  
# urbansound8k.validate() 

In [4]:
from wave_networks.prepare_data import *

path = "/homes/55/bwilop/gdl/ablation_study_wavelet_networks/calculated_data/preprocessed_data.pkl"
dataset_audio_timeseries = compute_dataset_audio_timeseries(path, urbansound8k)

/homes/55/bwilop/gdl/wavelet_networks/wavelet_networks/wavelet_networks_venv/lib64/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
print(dataset_audio_timeseries.train_data.shape)
print(dataset_audio_timeseries.validation_data.shape)
print(dataset_audio_timeseries.test_data.shape)

torch.Size([6273, 90000])
torch.Size([806, 90000])
torch.Size([1653, 90000])


: 

: 

In [6]:
from wave_networks.training import train_model
from wave_networks.architecture_definitions import *

model = StandardConvolutionalNetwork()
folder = "/homes/55/bwilop/gdl/ablation_study_wavelet_networks/trained_models"
train_parameters = {
    'batch_size': 8,
    'learning_rate': 1e-3,
    'weight_decay': 1e-4,
    'max_epochs': 100,
    'early_stopping_patience': 5,
}
model = train_model(model, "SmallCNN", dataset_audio_timeseries, train_parameters, folder)

ModuleNotFoundError: No module named 'jaxtyping'

: 

: 

: 